# 유튜브 크롤링 후 DB 적재 자동화

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import warnings
import psycopg2
import re

In [3]:
#스크롤 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,6000);")
    time.sleep(2)

# 해시태그 함수
def get_hashtag(outline_test) :
    # hash_tag가 안달려있는 영상이 있기때문에 오류 방지
    try:
        hash_tag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
        #print(hash_tag)
        hashtag_list.append(hash_tag)
            
    except Exception as e:
        hash_tag = ""
        #print(hash_tag)
        hashtag_list.append(hash_tag)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

뷔스티에 원피스, 롱패딩, 숏패딩, 롱코트, 숏코트, 잠옷, 홈웨어, 트위드자켓, 블레이저 자켓, 싱글 자켓, 크롭 자켓, 레더 자켓, 루즈핏, 테일러드 자켓, 린넨자켓, 반팔자켓, 라이더자켓, 오버핏자켓, 청자켓, 니트가디건, 볼레로, 아가일, 크롭가디건, 반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', 후드집업, 투웨이집업, 여름아우터, 간절기아우터, 겨울아우터, 기모맨투맨, 니트맨투맨, 크롭 맨투맨', '남자 맨투맨', '반집업', 후드티, 남자후드, 꾸안꾸, 플리스, 반팔티, 크롭티, 블라우스, 와이셔츠, 셔츠, 체크셔츠, 타이다이, 옥스퍼드셔츠, 남방, 슬리브티, 니트, 남자니트, 조끼, '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', '미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울', '겨울룩북', '겨울옷쇼핑', '겨울코디', '겨울하울'

keyword_list = ['블라인드', '롤스크린', '커튼', '러그', '카페트', '매트', '쿠션', '방석', '소파커버', '조명', 'led평판등', '천장등', '장스탠드', '단스탠드', '데스크스탠드', '무드등', '장식조명', '벽조명', '센서등', '형광등', '집꾸미기', '방꾸미기', '인테리어', '선반', '행거', '옷장', '의자', '화장대', '콘솔', '전신거울', '침구세트', '이불', '베개', '토퍼', '패드', '매트리스커버', '자취생가구', '원형테이블', '사각테이블', '게이밍테이블', '사무용책상', '신혼침대', '이케아', '신혼가구', '탁상거울', '티비다이', '템바보드', '오픈수납장', '혼수가구', '진열장', '장식장', '책꽂이', '와인셀러', '캐비닛', '식탁', '데스크', '콰트로 책상', '주방상판', '프래그마', '사이드책상', '베드테이블', '매쉬 의자', '사무용의자', '학생의자', '공부용의자', '중역의자', '스툴', '헤드형의자', '회전의자', '회의실의자', '팔걸이의자', '욕실매트', '주방매트', '놀이매트', '층간소음매트', '극세사매트', '현관매트', '소프트매트', '소음매트', '뽀송매트', '미끄럼방지매트', '게이밍의자', '발매트', '사계절매트', '메모리폼', '플라스틱서랍', '이동형서랍', ]

'책추천', '책리뷰', '책소개', '인생책', '필독서', '인문서적', '에세이', '여행 책', '라이트노벨 책', '자기계발 책'

, , '무지주선반', '벽걸이선반', '조립형선반', '무볼트선반', '철제선반', '폴딩선반', '슈랙선반', '화분선반', '대나무선반', '무타공선반', '아쿠아소파', '패브릭소파', '가죽소파', '카우치', '천연소파', '방수소파', '리클라이너', '쿠션소파', '스웨이드소파', '자코모소파', '좌식소파', '워셔블소파', '이태리소파', '슈퍼싱글', '퀸사이즈침대', '킹사이즈침대', '일체형침대', '호텔침대', '스프링침대', '방수커버', '헤드리스침대', '딜라이트침대', '헹거', '왕자행거', '드레스옷장', '스탠드옷장', '선반형옷장', '거울옷장', '시스템옷장', '고정식옷장', '이동식옷장', '붙박이장', '슬라이딩옷장', '무볼트옷장', '거울화장대', '조명화장대', '전신화장대', '좌식화장대', '서랍화장대', '로마화장대', '콘센트화장대', '원목화장대', '레트로화장대'

In [87]:
keyword_list = ['와이드서랍', '시스템서랍', '전자렌지선반', '칸칸이서랍', '친환경서랍', '다용도서랍', '레일서랍', '투명서랍']

In [91]:
warnings.filterwarnings(action='ignore')

driver = webdriver.Chrome('C:\chromedriver.exe')

for keyword in keyword_list :

    # 키워드를 검색한 유튜브 주소 접속
    url = 'https://www.youtube.com/results?search_query={}'.format(keyword)
    driver.get(url)

    #사이트 최대화
    driver.maximize_window()

    # 필터 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/div/ytd-toggle-button-renderer/a/tp-yt-paper-button').click()
    time.sleep(1)

    # 구분 -> 동영상 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[2]/ytd-search-filter-renderer[1]/a/div/yt-formatted-string').click()

    # 업로드 날짜 -> 올해 클릭
    driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[1]/div[2]/ytd-search-sub-menu-renderer/div[1]/iron-collapse/div/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[5]/a').click()


    # 스크롤 다운 50번 실행
    body = driver.find_element_by_tag_name('body')
    body.send_keys(Keys.PAGE_DOWN)

    for i in range(1,50):
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    name = soup.select('a#video-title')
    video_url = soup.select('a#video-title')

    video_num_list = [] #영상번호
    name_list = [] #영상제목
    url_list = [] #영상주소

    for i in range(len(name)):
        video_num_list.append(i+1)
        name_list.append(name[i].text.strip())

    for i in video_url:
        url_list.append('{}{}'.format('https://www.youtube.com',i.get('href')))

    print(keyword, '영상 개수: ', len(name_list))

    # 업로드일 & 해시태그 가져오기
    day_list = [] #업로드일
    hashtag_list = [] #해시태그

    for i in range(0,len(name_list)):

        print(i+1, end=' ')
        
        if url_list[i].split('/')[3] == 'shorts':  #쇼츠

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            
            driver.get(url_list[i])

            time.sleep(2)

            #더보기 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[1]/ytd-reel-video-renderer[1]/div[2]/ytd-reel-player-overlay-renderer/div[2]/div[1]/ytd-menu-renderer/yt-icon-button/button/yt-icon').click()
            time.sleep(2)

            #설명 클릭
            driver.find_element_by_xpath('/html/body/ytd-app/ytd-popup-container/tp-yt-iron-dropdown/div/ytd-menu-popup-renderer/tp-yt-paper-listbox/ytd-menu-service-item-renderer/tp-yt-paper-item').click()
            time.sleep(2)

            #업로드일 정보 가져오기
            day = driver.find_element_by_id("publish-time").text.split(':')[1].strip()
            day_list.append(day)

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = driver.find_element_by_class_name("style-scope ytd-reel-description-sheet-renderer").text
            get_hashtag(outline_test)

        else :

            driver.get(url_list[i])
                
            time.sleep(2)    

            html_source = driver.page_source
            soup = BeautifulSoup(html_source, 'html.parser')

            time.sleep(3)  

            #업로드일 정보 가져오기
            #day = soup.select('#formatted-snippet-text > span:nth-child(3)')[0].text
            day = driver.find_element_by_id("info-strings").text.split('\n')[0]
            day_list.append(day)

            if day == "" :
                print('error in getting day')
                break

            time.sleep(2) 

            # 해시태그 가져오기
            outline_test = soup.find("yt-formatted-string", {"class":"content style-scope ytd-video-secondary-info-renderer"}).text    
            get_hashtag(outline_test)

    print('끝')

    day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
    for day in day_list : 
        day = day.replace('.', '')
        day = day.replace('최초 공개: ', '')
        day = day.replace('실시간 스트리밍 시작일: ', '')
        day = day.replace(' ', '-')
        day_list2.append(day)

    #df화
    youtubeDic = {
        '제목': name_list,
        '업로드일': day_list2,
        '주소': url_list,
        '해시태그' : hashtag_list
    }

    youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
    youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
    youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

    print(youtubeDf.tail())


    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'yt_interior'

    #table에 데이터 넣기
    for i in range(len(youtubeDf)) :
        cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
        conn.commit()

    #db 연결 종료
    conn.cursor().close()
    conn.close()
    

이동형서랍 영상 개수:  151
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 끝
                                                    제목       업로드일  \
146  책상서랍으로 만든 2in 1 아름다운 돌화덕/DIY- Making Firewood ... 2021-08-20   
147              서랍안전 잠금장치,싱크대하부장 안전잠금장치, 안전잠금장치,Dajam 2022-02-22   
148           가성비 완전대박 이케아서랍장 상품 가격 후기 비교 추천판매순위 top10 2022-06-10   
149                [광고] #서랍장 추천 비교 순위 가성비 인기 구매 TOP 10 2021-07-24   
150                                       아지이 원목철제 서랍장 2022-03-25   

                                              주소  \
146  https://www.

StringDataRightTruncation: 오류:  character varying(500) 자료형에 너무 긴 자료를 담으려고 합니다.


In [51]:
for i in range(len(day_list)) :
    print(i, day_list[i], end=' / ')

day_list[70] = '2022-06-12'

0 2021. 6. 26. / 1 2022. 1. 4. / 2 2022. 2. 22. / 3 2022. 4. 4. / 4 2021. 10. 31. / 5 2021. 7. 23. / 6 2022. 4. 30. / 7 2021. 12. 29. / 8 2021. 12. 5. / 9 2022. 2. 24. / 10 2022. 3. 29. / 11 2022. 1. 14. / 12 2021. 12. 26. / 13 2022. 4. 22. / 14 2021. 7. 1. / 15 2021. 8. 20. / 16 2022. 3. 16. / 17 2021. 8. 3. / 18 2022. 1. 11. / 19 2021. 10. 26. / 20 2022. 2. 9. / 21 2021. 9. 27. / 22 2021. 8. 18. / 23 2022. 2. 16. / 24 2021. 8. 21. / 25 2022. 5. 7. / 26 2021. 7. 22. / 27 2022. 6. 2. / 28 2021. 10. 10. / 29 2021. 11. 4. / 30 2022. 1. 6. / 31 2021. 8. 31. / 32 2022. 3. 5. / 33 2021. 8. 23. / 34 2021. 8. 2. / 35 2021. 9. 11. / 36 2022. 2. 19. / 37 2021. 8. 3. / 38 2022. 2. 18. / 39 2021. 10. 26. / 40 2021. 11. 19. / 41 2021. 12. 11. / 42 2021. 11. 26. / 43 2021. 7. 2. / 44 2022. 3. 8. / 45 2021. 11. 4. / 46 2022. 3. 31. / 47 2022. 1. 27. / 48 2021. 12. 10. / 49 2022. 3. 20. / 50 2022. 5. 29. / 51 2022. 2. 27. / 52 2021. 11. 5. / 53 2022. 4. 29. / 54 2022. 2. 8. / 55 2022. 3. 4. / 56 2021

In [96]:
day_list2 = [] #전처리 후 업로드일

    #업로드일 날짜형식으로 통일화
for day in day_list : 
    day = day.replace('.', '')
    day = day.replace('최초 공개: ', '')
    day = day.replace('최초 공개일: ', '')
    day = day.replace('실시간 스트리밍 시작일: ', '')
    day = day.replace(' ', '-')
    day_list2.append(day)

    #df화
youtubeDic = {
    '제목': name_list,
    '업로드일': day_list2,
    '주소': url_list,
    '해시태그' : hashtag_list
}

youtubeDf = pd.DataFrame(youtubeDic)

    # datetime 타입으로 변환
youtubeDf['업로드일'] = pd.to_datetime(youtubeDf['업로드일'])

    # 중복값제거
youtubeDf = youtubeDf.drop_duplicates(['주소'], keep='first')

print(youtubeDf.tail())


    #postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'yt_interior'

    #table에 데이터 넣기
for i in range(len(youtubeDf)) :
    cur.execute(insert_command(table), (youtubeDf.iloc[i][0], youtubeDf.iloc[i][1], youtubeDf.iloc[i][2], youtubeDf.iloc[i][3]))
    conn.commit()

    #db 연결 종료
conn.cursor().close()
conn.close()

                                                    제목       업로드일  \
146  책상서랍으로 만든 2in 1 아름다운 돌화덕/DIY- Making Firewood ... 2021-08-20   
147              서랍안전 잠금장치,싱크대하부장 안전잠금장치, 안전잠금장치,Dajam 2022-02-22   
148           가성비 완전대박 이케아서랍장 상품 가격 후기 비교 추천판매순위 top10 2022-06-10   
149                [광고] #서랍장 추천 비교 순위 가성비 인기 구매 TOP 10 2021-07-24   
150                                       아지이 원목철제 서랍장 2022-03-25   

                                              주소  \
146  https://www.youtube.com/watch?v=VDYgpGw1DcU   
147  https://www.youtube.com/watch?v=Whm8_xL8L-o   
148  https://www.youtube.com/watch?v=haE77rTjuk8   
149  https://www.youtube.com/watch?v=oNITjPDf2QE   
150  https://www.youtube.com/watch?v=5qsD9zPUGFw   

                                          해시태그  
146                                             
147                                             
148  #이케아 #마켓비 #이케아 #큐물러스 #윤성디자인 #마켓비 #이케아 #한샘  
149                                             
150                        

In [95]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(hashtag_list)) :
    a = len(hashtag_list[i])

    if a > b :
        b = a
        print(i, b)

print(hashtag_list[77][:205])

1 15
6 69
12 97
15 115
77 201
#마스크정리함 #마스크수납함 #생활탐구수납함 #한샘리빙박스 #정리수납전문가 #옷정리노하우 #반함오픈수납함 #약정리함 #다이소수납 #서랍정리함 #악세사리트레이 #플라스틱박스 #책상미니서랍장 #서랍리빙박스 #서랍함 #수납통 #틈새수납함 #서랍장정리 #책상정리 #책상서랍정리 #욕실수납함 #플라스틱수납함 #수납박스 #서랍정리 #이동식정리함 #서랍트레이 #플라스틱상자


In [94]:
hashtag_list[77] = hashtag_list[77][:201]

# #############여기까지만#############

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [65]:
#db 연결 종료
conn.cursor().close()
conn.close()

create table yt_clothes (
	id serial,
	title varchar(100),
	date date,
	url varchar(200));